In [1]:
#导入包
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

ModuleNotFoundError: No module named 'gensim'

In [2]:
#读取数据集
train = pd.read_csv('./cnews/train.tsv',sep='\t',header=None,names=['label','content'])
val = pd.read_csv('./cnews/dev.tsv',sep='\t',header=None,names=['label','content'])
test = pd.read_csv('./cnews/test.tsv',sep='\t',header=None,names=['label','content'])

In [3]:
train.head()

,label,content
0,体育,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1,体育,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2,体育,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3,体育,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4,体育,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...


In [ ]:
jieba pkuseg

In [4]:
import jieba
def content_cut(x):
    x = jieba.lcut(x)
    x = " ".join(x)
    return x

In [5]:
train['content'] = train['content'].map(lambda x: content_cut(x))
val['content'] = val['content'].map(lambda x: content_cut(x))
test['content'] = test['content'].map(lambda x: content_cut(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hcq\AppData\Local\Temp\jieba.cache
Loading model cost 1.272 seconds.
Prefix dict has been built successfully.


In [6]:
df = pd.concat([train,val,test],axis=0)

In [9]:
df.head()

,label,content
0,体育,马晓旭 意外 受伤 让 国奥 警惕 无奈 大雨 格外 青睐 殷家 军 记者 傅亚雨 沈阳...
1,体育,商瑞华 首战 复仇 心切 中国 玫瑰 要 用 美国 方式 攻克 瑞典 多曼来 了 ， 瑞...
2,体育,冠军 球队 迎新 欢乐 派对 黄旭获 大奖 张军 赢 下 PK 赛 新浪 体育讯 12 ...
3,体育,辽足 签约 危机 引 注册 难关 高层 威逼利诱 合同 笑里藏刀 新浪 体育讯 2 月 ...
4,体育,揭秘 谢亚龙 被 带走 ： 总局 电话 骗局 复制 南杨 轨迹 体坛周报 特约记者 张锐...


# 训练word2vec

In [7]:
sentences = [document.split(' ') for document in df['content'].values]

In [8]:
model = Word2Vec(sentences=sentences,
     size=200,#维度
     alpha=0.025, #默认
     window=5, #默认
     min_count=2,#2，3
     sample=0.001,#
     seed=2018, #
     workers=11, #线程
     min_alpha=0.0001, 
     sg=0, #cbow
     hs=0, #负采样
     negative=5,#负采样个数
     ns_exponent=0.75, 
     cbow_mean=1,#求和再取平均
     iter=10, #10到20
     compute_loss =True
     )

In [ ]:
## 保存word2vec

In [10]:
model.save("./word2vec/word2vec_word_200")

In [11]:
model = Word2Vec.load("./word2vec/word2vec_word_200")

In [ ]:
model.vo

In [4]:
#查看单词的向量
model.wv['816903'].shape

(200,)

In [7]:
help(model.most_similar)

Help on method most_similar in module gensim.models.base_any2vec:

most_similar(positive=None, negative=None, topn=10, restrict_vocab=None, indexer=None) method of gensim.models.word2vec.Word2Vec instance
    Deprecated, use self.wv.most_similar() instead.
    
    Refer to the documentation for :meth:`~gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar`.



In [9]:
#和这个单词最相似的单词
model.most_similar("",topn=20)

/home/hcq/miniconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('12875', 0.8677932620048523),
 ('679169', 0.8625671863555908),
 ('90540', 0.841310977935791),
 ('425105', 0.8043540716171265),
 ('866203', 0.7445841431617737),
 ('122513', 0.7241939902305603),
 ('1234861', 0.7100560069084167),
 ('85838', 0.7024739980697632),
 ('1189755', 0.6224364638328552),
 ('426716', 0.5778474807739258),
 ('816903', 0.5615671873092651),
 ('797828', 0.557973325252533),
 ('1254728', 0.5530299544334412),
 ('11177', 0.546566367149353),
 ('850976', 0.5452205538749695),
 ('48896', 0.5422906875610352),
 ('903604', 0.5324429273605347),
 ('1146147', 0.5293028354644775),
 ('1200328', 0.527854859828949),
 ('1104318', 0.5183314085006714)]

In [10]:
#计算两个单词之间相似性
model.wv.similarity("816903","1226448")

0.6617146

In [ ]:
#看词表
model.wv.vocab.keys()

In [15]:
#迭代模型

In [ ]:
sentences_next=[]
for document in test['word_seg'].tolist():
    sentences_next.append(document.split(" "))

In [ ]:
model.train(sentences=sentences_next, total_examples=model.corpus_count,  epochs=model.iter) 

In [ ]:
model.save()
